This notebook is used to parse crsp data that only includes stocks that are ordinary/common shares and listed on the NYSE, AMEX, or NASDAQ. Also, we set negative prices equal to 'NA' since a negative price represents the midpoint of the bid-ask spread and we only want closing price values.

# CRSP Data

In [1]:
import pandas as pd
from pathlib import Path  
import datetime

In [40]:
data = pd.read_csv("crsp_1926_2020.csv")
data.date = pd.to_datetime(data.date)

In [41]:
remove_indexes = data[(data['SHRCD'] != 10) & (data['SHRCD'] != 11)].index
data.drop(remove_indexes, inplace=True)
remove_indexes = data[(data['EXCHCD'] != 1) & (data['EXCHCD'] != 3) & (data['EXCHCD'] != 3)].index
data.drop(remove_indexes, inplace=True)

data.loc[data["PRC"] < 0, "PRC"] = 0 
data.loc[data["RET"] == 'C', "RET"] = 0
data.loc[data["SHRCD"] == 10, "SHRCD"] = "Ordinary" 
data.loc[data["SHRCD"] == 11, "SHRCD"] = "Common" 
data.loc[data["EXCHCD"] == 1, "EXCHCD"] = "NYSE" 
data.loc[data["EXCHCD"] == 2, "EXCHCD"] = "AMEX" 
data.loc[data["EXCHCD"] == 3, "EXCHCD"] = "NASDAQ" 

In [42]:
data = data.dropna()
data = data.sort_values('date')
data = data.reset_index()
data = data.drop('index', axis=1)

In [43]:
lowest_date = datetime.datetime(2010, 1, 29)
remove_indexes = data[data['date'] < lowest_date].index
data.drop(remove_indexes, inplace=True)
data = data.reset_index()
data = data.drop('index', axis=1)

In [48]:
cd data

/Users/helensiavelis/Desktop/Caltech/Senior/Spring 2023/CS 145/CS-145/data


In [49]:
filepath = Path('monthly_stock.csv')  
data.to_csv(filepath)  

# FFM Data 

In [27]:
## fix ffm 
ffm = pd.read_excel("F-F_Research_Data_Factors.xlsx")

In [230]:
new_dates = []
for index, row in ffm.iterrows():
    curr_date = row['Date']
    new_date = curr_date[:4] + "-" + curr_date[4:-2] + "-" + curr_date[6:] 
    new_dates.append(new_date)
ffm['Date'] = new_dates
ffm = ffm.dropna()

In [231]:
ffm['Date'] = pd.to_datetime(ffm.Date)

In [232]:
lowest_date = datetime.datetime(2010, 1, 4)
remove_indexes = ffm[ffm['Date'] < lowest_date].index
ffm.drop(remove_indexes, inplace=True)

In [233]:
ffm

,Date,Mkt-RF,SMB,HML,RF
22127,2010-01-04,1.69,0.61,1.13,0.000
22128,2010-01-05,0.31,-0.64,1.24,0.000
22129,2010-01-06,0.13,-0.22,0.57,0.000
22130,2010-01-07,0.40,0.09,0.98,0.000
22131,2010-01-08,0.33,0.37,0.01,0.000
...,...,...,...,...,...
25433,2023-02-22,-0.04,0.55,-0.43,0.018
25434,2023-02-23,0.46,0.21,-0.23,0.018
25435,2023-02-24,-1.09,-0.49,1.26,0.018
25436,2023-02-27,0.31,0.15,-0.29,0.018


In [234]:
filepath = Path('monthly_stock_ffm.csv')  
ffm.to_csv(filepath)  

In [113]:
## map permno to stock ticker/name 
